In [1]:
# add default values for parameters here

In [2]:
# Parameters
product = "notebooks/cluster_bars.ipynb"


In [3]:
import __path__

In [4]:
from config import MAIN_EXPERIMENT


WIDTH = MAIN_EXPERIMENT.nb_args[product]
AFFILIATION_TH = MAIN_EXPERIMENT.affiliation_theshold
PREPROCESSED_FILE = MAIN_EXPERIMENT.preprocessed_file
MAIN_MODEL = MAIN_EXPERIMENT.main_model
PICTURES = MAIN_EXPERIMENT.pictures
GROUPS = MAIN_EXPERIMENT.groups

In [5]:
import json


with open(PREPROCESSED_FILE, 'r') as f:
    preprocessed = json.load(f)

In [6]:
from gensim.models import TfidfModel

from src.utils import load


lda, dictionary, corpus = load(MAIN_MODEL)
tfidf = TfidfModel(corpus, id2word=dictionary)

In [7]:
scores_raw = [0 for _ in range(len(GROUPS))]
for policy in preprocessed:
    for paragraph in policy:
        topics = [t for t, s in lda[tfidf[dictionary.doc2bow(paragraph)]] if s > AFFILIATION_TH]

        for g in GROUPS:
            for t in topics:
                if t in g['topics']:
                    scores_raw[g['id']] += 1
                    
total = sum(scores_raw)
print(f'{total=}')
scores_raw

total=45036


[3372, 3209, 3484, 1532, 17747, 3795, 5198, 1492, 2064, 3143]

In [8]:
keys = [g['name'] for g in GROUPS]

In [9]:
keys[:]

['Вопросы пользователя по политике безопасности',
 'Сторонние веб-сайты',
 'Особая аудитория',
 'Защита персональных данных',
 'Сбор персональных данных',
 'Сбор данных, позволяющих отслеживать поведения пользователя на веб-сайте',
 'Передача персональных данных третьим лицам',
 'Хранение персональных данных',
 'Персонализация и маркетинг',
 'Изменение политики безопасности']

In [10]:
from textwrap import wrap

from src.utils import resolve_group_name


names = [resolve_group_name(i, GROUPS) for i, _ in enumerate(scores_raw)]
names = ['<br>'.join(wrap(n, width=WIDTH)) for n in names]


scores = [s / total * 100 for s in scores_raw]
names

['Вопросы пользователя по политике<br>безопасности',
 'Сторонние веб-сайты',
 'Особая аудитория',
 'Защита персональных данных',
 'Сбор персональных данных',
 'Сбор данных, позволяющих<br>отслеживать поведения пользователя<br>на веб-сайте',
 'Передача персональных данных<br>третьим лицам',
 'Хранение персональных данных',
 'Персонализация и маркетинг',
 'Изменение политики безопасности']

In [11]:
import plotly.graph_objects as go

from config import COLORS_B, COLORS_D


fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=list(reversed(scores)), y=list(reversed(names)),
        orientation='h'))

fig.update_layout(
    font=dict(family='Times New Roman', size=20, color='black'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    yaxis={'side': 'right'},
    margin=dict(pad=10),
    height=75*len(names),
    width=1000)

fig.update_xaxes(
    title=dict(
        font=dict(family='Times New Roman', size=25, color='black'),
        text='Процент присутствия аспектов политик'), 
    zerolinecolor='lightgrey',
    gridcolor='lightgrey',
    autorange="reversed",
    showgrid=True, 
    tickangle=0,
    ticklen=10)

fig.update_yaxes(
    title=dict(
        font=dict(family='Times New Roman', size=25, color='black')), 
    zerolinecolor='lightgrey',
    gridcolor='lightgrey',
    showgrid=True, 
    tickangle=0, 
    ticklen=10)

fig.update_traces(marker_color=[COLORS_D[4]]*16)

fig

In [12]:
fig.write_image(f'{PICTURES}/cluster_bars.png')

In [13]:
import plotly.graph_objects as go

from config import COLORS_D
import pandas as pd
import plotly.express as px
import numpy as np


def round_retain_sum(x):
    x = np.array(x)*10 # We want 2 decimal precision
    N = np.round(np.sum(x)).astype(int)
    y = x.astype(int)
    M = np.sum(y)
    K = N - M 
    z = y-x 
    if K!=0:
        idx = np.argpartition(z,K)[:K]
        y[idx] += 1     
    return (y/10.).tolist()


df = pd.DataFrame({'y': [0] * len(list(names)), 'name':list(reversed(names)), 'value':list(reversed(round_retain_sum(scores)))})


fig = px.bar(df, y='y', x='value', color='name', orientation='h', text_auto=True, color_discrete_sequence=COLORS_D)

fig.update_xaxes(
    title=dict(
        font=dict(family='Times New Roman', size=25, color='black'),
        text='Процент присутствия аспектов политик'), 
    zerolinecolor='lightgrey',
    gridcolor='lightgrey',
    showgrid=True, 
    tickangle=0,
    ticklen=10)

fig.update_yaxes(
    title=dict(
        font=dict(family='Times New Roman', size=25, color='black')), 
    tickangle=0,
    ticklen=10)

fig.update_yaxes(showticklabels=False, title=None)
fig.update_xaxes(range=[0, 100.2])
fig.update_traces(width=0.2)

fig.update_layout(
    legend=dict(
        title=None,
        entrywidth=.4,
        entrywidthmode='fraction',
        orientation='h',
        yanchor="top",
        y=50000,
        xanchor="center",
        x=.5),
    autosize=False,
    font=dict(family='Times New Roman', size=20, color='black'),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    yaxis={'side': 'right'},
    colorway=COLORS_D,
    margin=dict(r=10, l=10, b=10, t=10, pad=10),
    height=len(names)*105/2,
    width=1000)

fig

In [14]:
fig.write_image(f'{PICTURES}/cluster_one_bar.png')